In [1]:
import numpy as np

In [6]:
Kmat = np.load("Chladni-Kmat.npy")

Implement centers,radii = gershgorin(A)

In [7]:
def gershgorin(A):
    n = len(A)
    centers = np.diag(A)
    radii = np.zeros(n)
    for i in range(n):
        radii[i] = np.sum(np.abs(A[i,:])) - np.abs(A[i,i])
    return centers, radii

Localize the eigenvalues of K, and report the disk centers and radii

In [8]:
gershgorin(Kmat)

(array([1.29292219e+05, 1.03041439e+05, 6.49675787e+04, 4.36124119e+04,
        3.62737515e+04, 3.79900994e+04, 2.41669711e+04, 1.16511587e+04,
        1.38650805e+04, 5.60054767e+03, 1.17303879e+03, 1.76077135e+03,
        2.88423061e+02, 8.68968906e+01, 1.38933458e+01]),
 array([4.22316467e+04, 5.69275189e+04, 3.11606709e+04, 1.85323487e+04,
        7.87051614e+03, 1.78547026e+04, 1.54147167e+04, 2.51204139e+03,
        5.50296953e+03, 1.19528329e+03, 3.41990248e+02, 4.01214726e+02,
        7.14173194e+01, 2.54029437e+00, 2.59094641e-01]))

Implement a function lambda = rayleigh_qt(A,x),

In [ ]:
def rayleigh_qt(A,x):
    pass

Implement a function x, k = power iterate(A,x0) for power iteration. Don’t forget to choose and implement a suitable convergence criterion

Test it by finding the largest eigenvalue of the example matrices. Report the eigenvalue found, the Rayleigh residual (the residual of the Raleigh quotient as a least-squares system), and the number k of iterations used. 

hat is the largest eigenvalue of K? Visualize your eigenfunction using show waves(x,basis set) to see the wave, and show nodes(x,basis set) to see where the sand will gather. The eigenfunction for the largest eigenvalue should have nodes along an 8 x 8 grid. 

Write a Rayleigh quotient iteration function x, k = rayleigh iterate(A,x0, shift0), which takes a matrix A, an initial vector x0, and an approximate eigenvalue shift0 as arguments and returns an eigenvector x together with the number k of iterations used. The multiplication by the inverse matrix should be implemented using either your own LUfactorization from as x = lu solve(A,y) or QR-factorization from as x = qr solve(A,y). NB: If you did not get either to work in previous weeks, you can use an in-built linear solver and make a small note that you do this.

Test it with the example matrices, and report the eigenvalues found, Rayleigh residual, and the number k of iterations used. 